Parte 1: Generación del conjunto de datos

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re
import os
import csv
import sklearn.metrics as metrics

In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 52.5 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 6.8 MB 58.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
tsv_file = open("lcp_single_test.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

In [4]:
label = 1
count = 0
data_raw = pd.DataFrame(columns = range(5))
for row in read_tsv:
  if label == 1:
    data_raw.columns = row
    label = 0
  else:
    data_raw.loc[count] = [row[0], row[1], row[2], row[3], row[4]]
    count += 1


In [5]:
data_raw.head(10)

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,"But he, beckoning to them with his hand to be ...",hand,0.0
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,"If I forget you, Jerusalem, let my right hand ...",hand,0.19736842105263158
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,"the ten sons of Haman the son of Hammedatha, t...",hand,0.2
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,Let your hand be lifted up above your adversar...,hand,0.2678571428571429
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,"Abimelech chased him, and he fled before him, ...",entrance,0.0
5,30F94FBDNRKF12C1PDZGZ0IRI9XTB1,bible,"for as he thinks about the cost, so he is.",cost,0.25
6,3JHB4BPSFK9JLT5CAPRP3PCEA2CQ9R,bible,"When his armor bearer saw that Saul was dead, ...",armor,0.02777777777777779
7,3Y40HMYLL1I1EIURUEH8TTVLKQ9XUM,bible,"Therefore put on the whole armor of God, that ...",armor,0.2
8,30Y6N4AHYPWV0KXTZKLRUB59XBPRDA,bible,"Hezekiah listened to them, and showed them all...",armor,0.2222222222222222
9,38O9DZ0A62NFBR1TA7YCNXN94XN26F,bible,"That first slaughter, which Jonathan and his a...",armor,0.3194444444444444


In [6]:
data = pd.DataFrame(columns=['text', 'complexity'])
count = 0
for i in range(0, data_raw.shape[0]):
  data.loc[count] = [data_raw['sentence'][count].lower() + ' [SEP] ' + data_raw['token'][count].lower(), \
                    data_raw['complexity'][count]]
  count += 1


In [7]:
data['text'][0]

'but he, beckoning to them with his hand to be silent, declared to them how the lord had brought him out of the prison. [SEP] hand'

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [9]:
# Preprocess, tokenize and vectorize texts and labels
def vectorize_text(s, max_length):

    # Unicode normalization
    #s = s.strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    # Remove undesired characters
    #s = re.sub(r'([.¿?()\'",;:$€])', r' \1 ', s)
    #s = re.sub(r"[^a-zA-Záéíóú.,!?;:()$€]+", r" ", s)

    # Text to tensor
    input_ids = tokenizer.encode(
      s,
      add_special_tokens=True,
      max_length=max_length,
      padding='longest', 
      truncation=True,
      return_tensors='np'
    )
    return input_ids[0]

In [10]:
data['text_vec'] = data.apply(lambda r: vectorize_text(r['text'], 512), axis=1)

In [11]:
data

,text,complexity,text_vec
0,"but he, beckoning to them with his hand to be ...",0.0,"[101, 2021, 2002, 1010, 10272, 13369, 2000, 20..."
1,"if i forget you, jerusalem, let my right hand ...",0.19736842105263158,"[101, 2065, 1045, 5293, 2017, 1010, 6744, 1010..."
2,"the ten sons of haman the son of hammedatha, t...",0.2,"[101, 1996, 2702, 4124, 1997, 10654, 2319, 199..."
3,let your hand be lifted up above your adversar...,0.2678571428571429,"[101, 2292, 2115, 2192, 2022, 4196, 2039, 2682..."
4,"abimelech chased him, and he fled before him, ...",0.0,"[101, 11113, 14428, 2571, 2818, 13303, 2032, 1..."
...,...,...,...
882,"the report by mr philippe busquin, on behalf o...",0.5694444444444444,"[101, 1996, 3189, 2011, 2720, 11169, 3902, 125..."
883,section v - court of auditors (sec(2002) 405 -...,0.5357142857142857,"[101, 2930, 1058, 1011, 2457, 1997, 20964, 201..."
884,"- mr president, on 1 june, on the basis of inf...",0.546875,"[101, 1011, 2720, 2343, 1010, 2006, 1015, 2238..."
885,i would like to extend a warm welcome to this ...,0.5714285714285714,"[101, 1045, 2052, 2066, 2000, 7949, 1037, 4010..."


In [12]:
# Separar los conjuntos de datos
data = data.sample(frac=1)
train_portion = 0.9
split_point = int(train_portion*len(data))
train_data, test_data =  data[:split_point], data[split_point:] 
print(len(train_data), 'train, ', len(test_data), 'test')

798 train,  89 test


In [13]:
#Create dataset en pytorch
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu()
        attention_mask = torch.ones([input_ids.size(0)]).cpu()
        targets = self.data.complexity.iloc[index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': targets
         }
            
    def __len__(self):
        return self.len

In [14]:
train_set, test_set = MyDataset(train_data), MyDataset(test_data)

In [15]:
test_set.data

,text,complexity,text_vec
69,"moses raised up the tabernacle, and laid its s...",0.18333333333333335,"[101, 9952, 2992, 2039, 1996, 21628, 11795, 18..."
420,"in contrast, null mutations of either nt-3 or ...",0.27380952380952384,"[101, 1999, 5688, 1010, 19701, 14494, 1997, 25..."
377,because of the current unsatisfactory options ...,0.40625,"[101, 2138, 1997, 1996, 2783, 4895, 16846, 248..."
871,"at the same time, let me welcome to the visito...",0.4852941176470589,"[101, 2012, 1996, 2168, 2051, 1010, 2292, 2033..."
698,but i would like to emphasise one thing: long-...,0.21875,"[101, 2021, 1045, 2052, 2066, 2000, 7902, 2063..."
...,...,...,...
863,draftsman. [SEP] draftsman,0.4305555555555556,"[101, 28967, 2386, 1012, 102, 28967, 2386, 102]"
583,"madam president, although i know that we have ...",0.203125,"[101, 21658, 2343, 1010, 2348, 1045, 2113, 200..."
327,affected infants exhibit a complete or near co...,0.1607142857142857,"[101, 5360, 16725, 8327, 1037, 3143, 2030, 237..."
428,"in this report, we have conducted detailed ana...",0.2857142857142857,"[101, 1999, 2023, 3189, 1010, 2057, 2031, 4146..."


In [16]:
train_set.data

,text,complexity,text_vec
212,"strangers, the terrible of the nations, have c...",0.6666666666666666,"[101, 12358, 1010, 1996, 6659, 1997, 1996, 374..."
292,morphologically detectable differences in the ...,0.1111111111111111,"[101, 24012, 2135, 11487, 3085, 5966, 1999, 19..."
459,normally cells accumulate h3-2mek9 and h3-3mek...,0.34090909090909094,"[101, 5373, 4442, 27598, 1044, 2509, 1011, 101..."
756,the next item on the agenda is the continuatio...,0.25,"[101, 1996, 2279, 8875, 2006, 1996, 11376, 200..."
405,it is possible that a subtle increase in m-ops...,0.33999999999999997,"[101, 2009, 2003, 2825, 2008, 1037, 11259, 362..."
...,...,...,...
245,jacob will break his clods. [SEP] clods,0.5735294117647058,"[101, 6213, 2097, 3338, 2010, 18856, 20620, 10..."
320,"additionally, none of these studies addressed ...",0.15625,"[101, 5678, 1010, 3904, 1997, 2122, 2913, 8280..."
251,"but i will stay at ephesus until pentecost, [S...",0.625,"[101, 2021, 1045, 2097, 2994, 2012, 4958, 1568..."
308,while cone photoreceptors are born during the ...,0.29166666666666663,"[101, 2096, 13171, 6302, 2890, 3401, 13876, 56..."


In [17]:
len(test_set)

89

In [18]:
len(train_set)
#len(test_set)

798

In [19]:
from transformers import BertForSequenceClassification, BertConfig
config = BertConfig.from_pretrained('bert-large-uncased')
config.problem_type='regression'
config.num_labels=1
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', config=config)


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [20]:
from transformers import Trainer, TrainingArguments

def collate_batch(batch):
    """ Optimize memory by setting all vectors in batch to a length equal to max
        length found
    """
    
    def pad_sequence(in_tensor, max_size):
        """ Fill tensor with zeros up to max_size
        """
        out_tensor = np.zeros(max_size)
        out_tensor[:in_tensor.size(0)] = in_tensor.numpy()
        return out_tensor
    
    batch_inputs = []
    batch_attention_masks = []
    batch_targets = []

    max_size = max([ex['input_ids'].size(0) for ex in batch])
    for item in batch:
        batch_inputs.append(pad_sequence(item['input_ids'], max_size))
        batch_attention_masks.append(pad_sequence(item['attention_mask'], max_size))
        batch_targets.append(float(item['labels']))
    return {
        "input_ids": torch.tensor(batch_inputs, dtype=torch.long),
        "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
        "labels": torch.tensor(batch_targets, dtype=torch.float)
    }

class MyTrainer(Trainer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def get_train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        collate_fn=collate_batch
    )

  def get_eval_dataloader(self, eval_dataset):
    return DataLoader(
        self.eval_dataset,
        collate_fn=collate_batch
    )


In [21]:
from transformers import EvalPrediction
from sklearn import metrics, feature_selection

## Para calcular nuestras propias métricas
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds)
    return {"MSE": ((preds - p.label_ids) ** 2).mean().item(),
            'R2': metrics.r2_score(p.label_ids, preds),
            'RMSE': metrics.mean_squared_error(p.label_ids, preds),
            'MAE': metrics.mean_absolute_error(p.label_ids, preds),
            'Poisson': metrics.mean_poisson_deviance(p.label_ids, preds)}
            #'Pearson': feature_selection.r_regression(p.label_ids, preds)}

In [22]:
training_args = TrainingArguments(
    output_dir='output',
    evaluation_strategy='steps',
    eval_steps=100,
    num_train_epochs=5,
    remove_unused_columns=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 798
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3990
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Step,Training Loss,Validation Loss,Mse,R2,Rmse,Mae,Poisson
100,No log,0.016369,0.016369,-0.015356,0.016369,0.093386,0.051416
200,No log,0.021649,0.021649,-0.342835,0.021649,0.102732,0.072975
300,No log,0.021377,0.021377,-0.325945,0.021377,0.101998,0.071758
400,No log,0.017076,0.017076,-0.059213,0.017076,0.101627,0.053558
500,0.081500,0.103253,0.103253,-5.404537,0.103253,0.303054,0.230904
600,0.081500,0.018771,0.018771,-0.164346,0.018771,0.110857,0.058554
700,0.081500,0.016224,0.016224,-0.006346,0.016224,0.093761,0.050905
800,0.081500,0.016152,0.016152,-0.001849,0.016152,0.094236,0.050654
900,0.081500,0.016213,0.016213,-0.005667,0.016213,0.093831,0.050867
1000,0.079800,0.020229,0.020229,-0.254733,0.020229,0.117739,0.062656


***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-10

TrainOutput(global_step=3990, training_loss=0.0462334052064365, metrics={'train_runtime': 1344.9528, 'train_samples_per_second': 2.967, 'train_steps_per_second': 2.967, 'total_flos': 320623349588100.0, 'train_loss': 0.0462334052064365, 'epoch': 5.0})